In [215]:
#!pip install pyarrow

In [216]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight
import os

*Загружает данные из Parquet файла и выполняет предобработку*

In [217]:
def prepocess(df):
    # пGопробую задать какие-то веса каждому feedback
    df['interaction_score'] = (
    df['timespent'] / 255.0 +  # Нормализую время просмотра
    df['like'] * 5 +
    df['dislike'] * (-2) +  # Дизлайк - негативный сигнал
    df['share'] * 10 +
    df['bookmark'] * 8 +
    df['click_on_author'] * 3 +
    df['open_comments'] * 2
    )
    
    # Убедимся, что итоговый score неотрицательный 
    df['interaction_score'] = df['interaction_score'].clip(lower=0.1)
    
    # Для ALS нужны целочисленные идентификаторы, начинающиеся с 0.
    # Создаем маппинги для user_id и item_id.
    df['user_id_mapped'], user_map = pd.factorize(df['user_id'])
    df['item_id_mapped'], item_map = pd.factorize(df['item_id'])
    
    print(f"Уникальных пользователей: {len(user_map)}")
    print(f"Уникальных клипов: {len(item_map)}")
    
    return df, user_map, item_map


In [218]:
def load_and_preprocess_data(file_path):

    print("Загрузка данных...")
    df = pd.read_parquet(file_path)
    print(f"Данные загружены. Форма: {df.shape}")
    
    return prepocess(df)

In [219]:
df, user_map, item_map = load_and_preprocess_data('train.parquet')
df.head()

Загрузка данных...
Данные загружены. Форма: (1904581, 12)
Уникальных пользователей: 8531
Уникальных клипов: 3729


,user_id,item_id,place,platform,agent,timespent,like,dislike,share,bookmark,click_on_author,open_comments,interaction_score,user_id_mapped,item_id_mapped
0,141827770,160593626,1,1,1,56,False,False,False,False,False,False,5.6,0,0
1,494341617,402842289,1,0,0,11,False,False,False,False,False,False,1.1,1,1
2,453833313,102672169,0,0,0,45,False,False,False,False,False,False,4.5,2,2
3,154047442,139795075,1,0,0,39,False,False,False,False,False,False,3.9,3,3
4,330326695,232077218,1,0,0,57,False,False,False,False,False,False,5.7,4,4


*Строит user-item matrix*

In [220]:
def build_interaction_matrix(df, num_users, num_items):

    print("Построение матрицы взаимодействий...")
    # interaction_score как сила взаимодействия
    interaction_matrix = sp.csr_matrix(
        (df['interaction_score'], (df['user_id_mapped'], df['item_id_mapped'])),
        shape=(num_users, num_items)
    )
    print(f"Размерность матрицы взаимодействий: {interaction_matrix.shape}")
    print(f"Заполненность матрицы: {interaction_matrix.nnz / (num_users * num_items):.6f}")
    return interaction_matrix


In [221]:
interaction_matrix = build_interaction_matrix(df, len(user_map), len(item_map))

Построение матрицы взаимодействий...
Размерность матрицы взаимодействий: (8531, 3729)
Заполненность матрицы: 0.059870


*Обучает модель ALS на матрице взаимодействий*

In [222]:
def train_als_model(interaction_matrix, FACTORS=100, REGULARIZATION=0.01, ITERATIONS=10, RANDOM_STATE=42, gpu=False):

    print("Обучение модели ALS...")
    
    # Применяем взвешивание BM25, чтобы снизить влияние очень активных пользователей
    weighted_matrix = bm25_weight(interaction_matrix, K1=2, B=0.8)
    
    # Инициализируем и обучаем модель
    model = AlternatingLeastSquares(
        factors=FACTORS,
        regularization=REGULARIZATION,
        iterations=ITERATIONS,
        random_state=RANDOM_STATE,
        use_gpu=gpu
    )
    
    model.fit(weighted_matrix)
    print("Обучение завершено.")
    return model

In [223]:
model = train_als_model(interaction_matrix)

Обучение модели ALS...


c:\Users\limon\AppData\Local\Programs\Python\Python311\Lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.01639866828918457 seconds
  warnings.warn(
100%|██████████| 10/10 [00:18<00:00,  1.86s/it]

Обучение завершено.


*Генерирует рекомендации для всех пользователей.*

In [ ]:
def generate_recommendations(model, interaction_matrix, user_map, item_map, top_n=10):
    print(f"Генерация {top_n} рекомендаций для каждого пользователя...")
    
    user_ids = np.arange(len(user_map))
    item_ids, scores = model.recommend(
        user_ids, 
        interaction_matrix[user_ids], 
        N=top_n, 
        filter_already_liked_items=True # Исключаем items,  с которыми user уже взаимодейвствовал
    )
    
    recommendations = []
    for internal_user_id, (internal_item_ids, _) in enumerate(zip(item_ids, scores)):
        original_user_id = user_map[internal_user_id]
        for internal_item_id in internal_item_ids:
            if internal_item_id != -1:  
                original_item_id = item_map[internal_item_id]
                recommendations.append({
                    'user_id': original_user_id,
                    'recs': original_item_id
                })
    
    return recommendations

In [225]:
# Реки для train data
train_recs = generate_recommendations(model, interaction_matrix, user_map, item_map)

Генерация 10 рекомендаций для каждого пользователя...


In [226]:
# Реки test data
# Загрузка и предобработка
df_test, user_map_test, item_map_test = load_and_preprocess_data('test.parquet')

print(f'всего {len(user_map)} различных users и {len(item_map)} разных items')
print(f'всего {len(user_map_test)} различных users и {len(item_map_test)} разных items в test')
print(f'всего совпадают {len(set(user_map) & set(user_map_test))} users и {len(set(item_map) & set(item_map_test ))} разлных items в train  и test выборках')

Загрузка данных...
Данные загружены. Форма: (1309342, 12)
Уникальных пользователей: 8528
Уникальных клипов: 6246
всего 8531 различных users и 3729 разных items
всего 8528 различных users и 6246 разных items в test
всего совпадают 8528 users и 3323 разлных items в train  и test выборках


Все test users есть в train, но не все items есть в train. Проблема холодного старта. Items,  которых нет в train не будут в топ 10 релевантных.

In [227]:
# Проверяем наличие каждого item_id из test_df в train_df
mask = ~df_test['item_id'].isin(df['item_id'])

# Оставляем только те строки, где item_id содержится в train_df
filtered_test_df = df_test[~mask]

In [228]:
filtered_test_df, filtered_test_user_map, filtered_test_item_map = prepocess(filtered_test_df)

Уникальных пользователей: 8527
Уникальных клипов: 3323


C:\Users\limon\AppData\Local\Temp\ipykernel_1768\3256959224.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['interaction_score'] = (
C:\Users\limon\AppData\Local\Temp\ipykernel_1768\3256959224.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['interaction_score'] = df['interaction_score'].clip(lower=0.1)
C:\Users\limon\AppData\Local\Temp\ipykernel_1768\3256959224.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [229]:
print(f'всего совпадают {len(set(user_map) & set(user_map_test))} users и {len(set(item_map) & set(item_map_test ))} разлных items в train  и test выборках')

всего совпадают 8528 users и 3323 разлных items в train  и test выборках


In [230]:
filtered_test_interaction_matrix = build_interaction_matrix(filtered_test_df, len(filtered_test_user_map), len(filtered_test_item_map))

Построение матрицы взаимодействий...
Размерность матрицы взаимодействий: (8527, 3323)
Заполненность матрицы: 0.023139


In [231]:
new_mod = train_als_model(filtered_test_interaction_matrix)

c:\Users\limon\AppData\Local\Programs\Python\Python311\Lib\site-packages\implicit\utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.007515907287597656 seconds
  warnings.warn(


Обучение модели ALS...


100%|██████████| 10/10 [00:13<00:00,  1.36s/it]

Обучение завершено.


In [235]:
filtered_test_recs = generate_recommendations(new_mod, filtered_test_interaction_matrix, filtered_test_user_map, filtered_test_item_map)

Генерация 10 рекомендаций для каждого пользователя...


In [237]:
recommendations_df = pd.DataFrame(filtered_test_recs)
recommendations_df.to_csv('recs.csv', index=False, sep=',')
    